In [1]:
import json
import math
import uuid
import random
import pandas as pd
from datetime import datetime
from dateutil import tz
from dateutil import parser as dateparser
from faker import Faker
import fhirclient.r3.models.patient as p
import fhirclient.r3.models.humanname as hn
import fhirclient.r3.models.observation as o
import fhirclient.r3.models.quantity as q
import fhirclient.r3.models.coding as c
import fhirclient.r3.models.codeableconcept as cc
import fhirclient.r3.models.fhirdate as dt
import fhirclient.r3.models.bundle as b
import fhirclient.r3.models.fhirreference as fr
import fhirclient.r3.models.identifier as iden

# initialize fake
fake = Faker()

# read the csv file
file = "loinc_2.csv"
df = pd.read_csv(file)
df.head(5)

,LOINC Code,LOINC Concept Name,Value[x],value,q.unit,q.system,q.code,cc.code,cc.system,cc.display,q.low,q.high,Unnamed: 12
0,26508-2,Band form neutrophils/100 leukocytes in Blood,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.0,6.0,https://healthmatters.io/understand-blood-test...
1,35332-6,Band form neutrophils/100 leukocytes in Blood ...,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.0,6.0,https://healthmatters.io/understand-blood-test...
2,764-1,Band form neutrophils/100 leukocytes in Blood ...,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.0,6.0,https://healthmatters.io/understand-blood-test...
3,30180-4,Basophils/100 leukocytes in Blood,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.5,1.0,https://www.ucsfhealth.org/medical-tests/blood...
4,706-2,Basophils/100 leukocytes in Blood by Automated...,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.5,1.0,https://www.ucsfhealth.org/medical-tests/blood...


In [2]:
patient = p.Patient()
gender = random.choice(['male','female'])
patient.gender = gender
name = hn.HumanName()
if gender == 'male':
    given_name = fake.first_name_male()
elif gender == 'female':
    given_name = fake.first_name_female()
    
family_name = fake.last_name()
name.family = family_name
name.given = [given_name, "Dash"]
name.text = given_name + " Dash " + family_name
name.use = 'official'
patient.name = [name]

birthdate = dt.FHIRDate()
birthdate.date = fake.date_between(start_date='-80y', end_date='-6y')
patient.birthDate = birthdate

identifier = iden.Identifier()
identifier.system = "http://example.org"
identifier.value = uuid.uuid4().hex
typecoding = c.Coding()
typecoding.system = 'http://hl7.org/fhir/v2/0203'
typecoding.code = 'MR'
typecoding.display = 'Medical record number'
itype = cc.CodeableConcept()
itype.coding = [typecoding]
identifier.type = itype
patient.identifier = [identifier]

patient.uuid = uuid.uuid4().urn
print(json.dumps(patient.as_json(), indent=2))
print(patient.uuid)
#import pprint as pp
#pp.pprint(patient.__dict__)
print(type(patient))

{
  "birthDate": "1985-12-06",
  "gender": "female",
  "identifier": [
    {
      "system": "http://example.org",
      "type": {
        "coding": [
          {
            "code": "MR",
            "display": "Medical record number",
            "system": "http://hl7.org/fhir/v2/0203"
          }
        ]
      },
      "value": "0eff80ed6dd14787a9391ea567e10b4a"
    }
  ],
  "name": [
    {
      "family": "Knox",
      "given": [
        "Becky",
        "Dash"
      ],
      "text": "Becky Dash Knox",
      "use": "official"
    }
  ],
  "resourceType": "Patient"
}
urn:uuid:c28f1181-94b8-4527-a742-2a02049ae5ae
<class 'fhirclient.r3.models.patient.Patient'>


In [3]:
bundle = b.Bundle()
bundle.type = 'transaction'
bundleEntry = b.BundleEntry()
bundleEntry.fullUrl = patient.uuid
bundleEntry.resource = patient
bundleEntryRequest = b.BundleEntryRequest()
bundleEntryRequest.method = "POST"
bundleEntryRequest.url = "Patient"
bundleEntry.request = bundleEntryRequest
bundle.entry = [bundleEntry]
print(json.dumps(bundle.as_json(), indent=2))

{
  "entry": [
    {
      "fullUrl": "urn:uuid:c28f1181-94b8-4527-a742-2a02049ae5ae",
      "request": {
        "method": "POST",
        "url": "Patient"
      },
      "resource": {
        "birthDate": "1985-12-06",
        "gender": "female",
        "identifier": [
          {
            "system": "http://example.org",
            "type": {
              "coding": [
                {
                  "code": "MR",
                  "display": "Medical record number",
                  "system": "http://hl7.org/fhir/v2/0203"
                }
              ]
            },
            "value": "0eff80ed6dd14787a9391ea567e10b4a"
          }
        ],
        "name": [
          {
            "family": "Knox",
            "given": [
              "Becky",
              "Dash"
            ],
            "text": "Becky Dash Knox",
            "use": "official"
          }
        ],
        "resourceType": "Patient"
      }
    }
  ],
  "type": "transaction",
  "resourceType": "Bu

In [4]:
subject = fr.FHIRReference()
subject.reference = patient.uuid
print(json.dumps(subject.as_json(), indent=2))

{
  "reference": "urn:uuid:c28f1181-94b8-4527-a742-2a02049ae5ae"
}


In [5]:

obslist = []
for index, row in df.iterrows():
    if row['Value[x]'] == 'Quantity':
        # for each LOINC code
        for i in range(random.randint(1, 10)):
            # make 1 to 10 examples
            observation = o.Observation()
            observation.status = "final"
        
            code = cc.CodeableConcept()
            codecode = c.Coding()
            codecode.system = 'http://loinc.org'
            codecode.code = row['LOINC Code']
            codecode.display = row['LOINC Concept Name']
            code.text = row['LOINC Concept Name']
            code.coding = [codecode]
            observation.code = code

            category = cc.CodeableConcept()
            categorycoding = c.Coding()
            categorycoding.system = "http://loinc.org"
            categorycoding.code = "laboratory"
            categorycoding.display = "Laboratory"
            category.coding = [categorycoding]
            observation.category = [category]

            observation.subject = subject
        
            # print(type(row['value']))
            
            effectiveDateTime = dt.FHIRDate()
            effectiveDateTime.date = fake.date_time_between(start_date=birthdate.date, end_date='now', tzinfo=tz.UTC)
            # effectiveDateTime.date = fake.date_time_between(start_date='-10y', end_date='now')
            observation.effectiveDateTime = effectiveDateTime
            
            value = row['value']
            #low = value * 0.1
            #high = value * 10
            low = row['q.low']
            high = row['q.high']
            newvalue = math.ceil(random.uniform(low, high)*10)/10
#             if value < 10.0:
#                 newvalue = math.ceil(random.uniform(0.1, 10.0)*10)/10
#             else:
#                 newvalue = math.ceil(random.uniform(1.0, 100.0)*10)/10
            # print(newvalue)
            # newvalue = 1.5
            valuequantity = q.Quantity()
            valuequantity.value = newvalue
            # print(valuequantity.value)
            valuequantity.unit = row['q.unit']
            valuequantity.system = row['q.system']
            valuequantity.code = row['q.code']
            # print(json.dumps(valuequantity.as_json(), indent=4))
            observation.valueQuantity = valuequantity
            
            # print(json.dumps(observation.as_json(), indent=4))
            bundleEntry = b.BundleEntry()
            bundleEntryRequest = b.BundleEntryRequest()
            bundleEntryRequest.method = "POST"
            bundleEntryRequest.url = "Observation"
            bundleEntry.request = bundleEntryRequest
            bundleEntry.resource = observation
            # myuuid = uuid.uuid4().urn
            bundleEntry.fullUrl = uuid.uuid4().urn
            # print(json.dumps(bundleEntry.as_json(), indent=4))
            # obslist.append(bundleEntry)
            bundle.entry.append(bundleEntry)      
# bundle.entry.append(obslist)
print(json.dumps(bundle.as_json(), indent=2))

{
  "entry": [
    {
      "fullUrl": "urn:uuid:c28f1181-94b8-4527-a742-2a02049ae5ae",
      "request": {
        "method": "POST",
        "url": "Patient"
      },
      "resource": {
        "birthDate": "1985-12-06",
        "gender": "female",
        "identifier": [
          {
            "system": "http://example.org",
            "type": {
              "coding": [
                {
                  "code": "MR",
                  "display": "Medical record number",
                  "system": "http://hl7.org/fhir/v2/0203"
                }
              ]
            },
            "value": "0eff80ed6dd14787a9391ea567e10b4a"
          }
        ],
        "name": [
          {
            "family": "Knox",
            "given": [
              "Becky",
              "Dash"
            ],
            "text": "Becky Dash Knox",
            "use": "official"
          }
        ],
        "resourceType": "Patient"
      }
    },
    {
      "fullUrl": "urn:uuid:48acfda1-9159-492

In [6]:
import pprint 
pp = pprint.PrettyPrinter(indent=2)
bundlejson = json.dumps(bundle.as_json())
# newlist = [x for x in bundle.entry if 'Observation' in type(x.resource)]
resourcetype = type(bundle.entry[1].resource)
# print(str(resourcetype))
if "Observation" in str(resourcetype):
    print("Observation")
# print(bundlejson)
# print(json.loads(bundlejson))
# pp.pprint(json.loads(bundlejson))
bundledict = json.loads(bundlejson)
# print(dict.keys(json.loads(bundlejson)))
# pp.pprint(bundledict['entry'][0])
# pp.pprint(bundledict['entry'][0]['resource']['resourceType'])
# bundleobs = lamba x: bundledict['entry'][x] if bundledict['entry'][x][re]
bundleobs = []
for i in bundledict['entry']:
    if i['resource']['resourceType'] == "Observation":
        bundleobs.append(i)
# pp.pprint(bundleobs[0])
print(dateparser.isoparse(bundleobs[0]['resource']['effectiveDateTime']))

tem = sorted(bundleobs, key = lambda x: dateparser.isoparse(x['resource']['effectiveDateTime']), reverse=True)
print("type of tem = " + str(type(tem)))
pp.pprint(tem[0])

Observation
2011-11-10 04:06:23+00:00
type of tem = <class 'list'>
{ 'fullUrl': 'urn:uuid:d581ee15-9000-4bfc-a14a-21da67fda3d4',
  'request': {'method': 'POST', 'url': 'Observation'},
  'resource': { 'category': [ { 'coding': [ { 'code': 'laboratory',
                                              'display': 'Laboratory',
                                              'system': 'http://loinc.org'}]}],
                'code': { 'coding': [ { 'code': '790-6',
                                        'display': 'Erythrocytes [#/volume] in '
                                                   'Blood by Manual count',
                                        'system': 'http://loinc.org'}],
                          'text': 'Erythrocytes [#/volume] in Blood by Manual '
                                  'count'},
                'effectiveDateTime': '2021-06-09T13:28:57Z',
                'resourceType': 'Observation',
                'status': 'final',
                'subject': { 'reference': 'u

In [7]:
# bundle = b.Bundle()
# bundle.type = 'transaction'
# bundleEntry = b.BundleEntry()
# bundleEntry.fullUrl = patient.uuid
# bundleEntry.resource = patient
# bundleEntryRequest = b.BundleEntryRequest()
# bundleEntryRequest.method = "POST"
# bundleEntryRequest.url = "Patient"
# bundleEntry.request = bundleEntryRequest
# bundle.entry = [bundleEntry]
# print(json.dumps(bundle.as_json(), indent=2))

newBundleEntry = b.BundleEntry()
newBundleEntry.fullUrl = patient.uuid
newBundleEntry.resource = patient
newBundleEntryRequest = b.BundleEntryRequest()
newBundleEntryRequest.method = "POST"
newBundleEntryRequest.url = "Patient"
newBundleEntry.request = newBundleEntryRequest
# print(json.dumps(newBundleEntry.as_json(), indent=2))
# print(" ")

bundle.entry = [newBundleEntry]
bundle.entry.extend(tem)
# print(json.dumps(bundle.as_json(), indent=2))
# bundle.entry.append(tem)
# print(json.dumps(bundle.as_json(), indent=2))
# pp.pprint(bundle['entry'][0:2])

# bundle.entry = newBundleEntry
print(json.dumps(bundle.as_json(), indent=2))

{
  "entry": [
    {
      "fullUrl": "urn:uuid:c28f1181-94b8-4527-a742-2a02049ae5ae",
      "request": {
        "method": "POST",
        "url": "Patient"
      },
      "resource": {
        "birthDate": "1985-12-06",
        "gender": "female",
        "identifier": [
          {
            "system": "http://example.org",
            "type": {
              "coding": [
                {
                  "code": "MR",
                  "display": "Medical record number",
                  "system": "http://hl7.org/fhir/v2/0203"
                }
              ]
            },
            "value": "0eff80ed6dd14787a9391ea567e10b4a"
          }
        ],
        "name": [
          {
            "family": "Knox",
            "given": [
              "Becky",
              "Dash"
            ],
            "text": "Becky Dash Knox",
            "use": "official"
          }
        ],
        "resourceType": "Patient"
      }
    },
    {
      "fullUrl": "urn:uuid:d581ee15-9000-4bf

In [8]:
with open('loinc_bundle8.json', 'w') as outfile:
    json.dump(bundle.as_json(), outfile, indent=2)

***
## Scratchpad below

In [46]:
data = [
    {"name": "Shawn", "date": "2019-12-07"}, 
    {"name": "John", "date": "2019-12-05"}, 
    {"name": "Dave", "date": "2019-12-01"}
]
print(data)
tem = sorted(
    data,
    key = lambda x: datetime.strptime(x["date"], '%Y-%m-%d')
)
print(tem)

[{'name': 'Shawn', 'date': '2019-12-07'}, {'name': 'John', 'date': '2019-12-05'}, {'name': 'Dave', 'date': '2019-12-01'}]
[{'name': 'Dave', 'date': '2019-12-01'}, {'name': 'John', 'date': '2019-12-05'}, {'name': 'Shawn', 'date': '2019-12-07'}]


In [8]:
uuid.uuid4().urn

'urn:uuid:be4cc3f4-2799-47b9-9350-8ff924036a2a'

In [9]:
fake = Faker()

In [10]:
fake.name()

'David Johnson'

In [11]:
fake.address()

'025 Katie Plains\nRollinsview, VT 53493'

In [12]:
for _ in range(10):
    print(fake.name())

Mr. Joseph Dougherty
Dr. Cheryl Henderson
Lisa Hill
Jennifer Lee
Francis Castro
Michael Jimenez
Christopher Choi II
Nicole Fowler
Lawrence Mccann
Edward Jackson


In [13]:
fake.date_time_between(start_date='-5y', end_date='now').isoformat()

'2020-11-17T14:27:40'

In [20]:
from dateutil import tz
# tzgmt = tz.gettz('UTC')
print(tzgmt)

tzfile('/usr/share/zoneinfo/UTC')


In [22]:
fakedate = fake.date_time_between(start_date='-10y', end_date='now', tzinfo=tz.UTC).isoformat()
print(fakedate)

2016-03-13T04:50:26+00:00


In [15]:
edt = dt.FHIRDate()
# print(fakedate)
# edt.date = datetime.fromisoformat('2011-11-04T00:05:23+00:00')
edt = datetime.fromisoformat(fakedate)
print(edt)

2014-04-29 10:40:12


In [16]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-07-08T08:04:50.951799


In [17]:
myfakedate = fake.date_time_between(start_date='-5y', end_date='now').isoformat()
myfakedate
datetime.fromisoformat(myfakedate).isoformat()

'2019-06-12T09:36:28'

In [18]:
import random
my_list = ['A'] * 20 + ['B'] * 20 + ['C'] * 60
random.choice(my_list)

'C'

In [20]:
mybun = b.Bundle()
mybun.type = 'transaction'
mybun.entry = []
for i in range(1,5):
    myobs = o.Observation()
    observation = o.Observation()
    observation.status = "final"

    code = cc.CodeableConcept()
    codecode = c.Coding()
    codecode.system = 'http://loinc.org'
    codecode.code = '30180-4'
    code.coding = [codecode]
    observation.code = code

    valuequantity = q.Quantity()
    valuequantity.value = i
    observation.valueQuantity = valuequantity
    
    bundleEntry = b.BundleEntry()
    bundleEntryRequest = b.BundleEntryRequest()
    bundleEntryRequest.method = "POST"
    bundleEntryRequest.url = "Observation"
    bundleEntry.request = bundleEntryRequest
    bundleEntry.resource = observation
    # print(json.dumps(bundleEntry.as_json(), indent=4))
    mybun.entry.append(bundleEntry)
    # print(type(mybun.entry))
    
print(json.dumps(mybun.as_json(), indent=4))
    

{
    "entry": [
        {
            "request": {
                "method": "POST",
                "url": "Observation"
            },
            "resource": {
                "code": {
                    "coding": [
                        {
                            "code": "30180-4",
                            "system": "http://loinc.org"
                        }
                    ]
                },
                "status": "final",
                "valueQuantity": {
                    "value": 1
                },
                "resourceType": "Observation"
            }
        },
        {
            "request": {
                "method": "POST",
                "url": "Observation"
            },
            "resource": {
                "code": {
                    "coding": [
                        {
                            "code": "30180-4",
                            "system": "http://loinc.org"
                        }
                    ]
       

In [21]:
gender = random.choice(['male','female'])
print(gender)

male


In [22]:
birthdate = fake.date()
print(birthdate)

2002-12-09


In [23]:
uuid.uuid4().hex

'3c12d477f3a145c4a5c1bf293b223850'